In [2]:
def run_TVDL_strategy(system_message, training_data, iterate_prompt,context_prompt, conversation_dropdown):
    class AssistantResponse:
        def __init__(self, role, content):
            self.role = role
            self.content = content
    # Initialize list to store predicted strengths and corresponding formulations
    predictions = []
    unique_formulations = []

    # Start with the system message
    messages = [
        {"role": "system", "content": system_message}
    ]
    
    # Add training data and iteration prompt
    if training_data:
        messages.append({"role": "assistant", "content": "Previously, we have tested these formulations with the following result:\n" + "\n".join(training_data)})
        messages.append({"role": "user", "content": iterate_prompt})
        training_formulations = extract_formulations_from_training_data(training_data)
    # Add the new user prompt to ask for three unique formulations
    messages.append({"role": "user", "content": "Output 3 three different unique formulations with a extremly high expected compressive strength. Make sure they lie on the allowed parameter grid but have not been tested previously."})
    
    # Make API call
    assistant_response = call_openai_api(messages, temp)
    
    # Extract the three formulations
    response_lines = assistant_response.choices[0].message.content.split('\n')
    # Define a regular expression pattern to capture the relevant information
    pattern = re.compile(r'Powderkg\s*=\s*(\d+),\s*wc\s*=\s*(\d+\.\d+),\s*materials\s*=\s*(\d+\.\d+/\d+\.\d+),\s*curing\s*=\s*(\w+)', re.IGNORECASE)
    
    # Use list comprehension to find all matches in the response lines
    formulations = [match.group(0) for line in response_lines for match in [pattern.search(line)] if match]
    #formulations = [line for line in response_lines if line.startswith("The formulation is")]
    if training_data:
        unique_formulations = [f for f in formulations if f not in training_formulations]
        if len(unique_formulations) == 0:
            print('no new formulations extracted')
        formulations = unique_formulations        
    # Handle the case where all suggested formulations are duplicates
    # Maybe prompt the assistant again or take other actions

    if conversation_dropdown.value == 'Complete':
        print('####', response_lines)
        print("Extracted Formulations:", formulations)
    
    # Prepare the forward task prompt

    forward_prompt_base = f"{context_prompt} {training_data} You are a powerful concrete strength prediction model. You can take into account design rules and the outcome of previous lab validation to predict the compressive strength of untested novel formulations. Answer in this exact format with a single number only 'The Predicted Strength is {{your Answer}} MPa.'"
    
    for formulation in formulations:
        # Create the full forward task prompt
        forward_prompt = f"{forward_prompt_base}\n Given these examples, what is the compressive strength of {formulation}?"
        
        if conversation_dropdown.value == 'Complete':
            print("Forward Prompt:", forward_prompt)
        
        # Make the API call to get the forward task prediction
        forward_response = call_openai_api([{"role": "system", "content": system_message}, {"role": "user", "content": forward_prompt}], 0)
        
        # Extract the predicted strength from the model's response
        try:
            predicted_strength_str = forward_response.choices[0].message.content.split(' ')[-2]
            predicted_strength = float(predicted_strength_str)
        except ValueError:
            print(f"Could not convert predicted strength for {formulation} to float. Skipping this formulation.")
            continue
        
        if conversation_dropdown.value == 'Complete':
            print(f"Predicted Strength for {formulation}: {predicted_strength} MPa")
        
        # Append to the predictions list
        predictions.append((formulation, predicted_strength))
    
    # Handle case where predictions list is empty
    if not predictions:
        print("No valid predictions were made. Defaulting to the first formulation.")
        return {'role': 'assistant', 'content': formulations[0] if formulations else 'Unable to make valid predictions.'}
    
    # Select the formulation with the highest predicted strength
    final_response, best_strength = max(predictions, key=lambda x: x[1])
    
    # Return an instance of AssistantResponse
    return AssistantResponse('assistant', final_response)

# Iterate over the different temperature settings
for temp in temperatures:

    # Repeat the whole experiment n times
    for experiment in range(NrOfExper):
        print(f"\n---\nStarting experiment {experiment+1}...\n---")
        training_data = []
        current_strength = 0.0
        iterations = 0
        formulations=[]
        # System message including both the role prompt and context
        system_message = system_role_prompt.value  + '\n' + context_prompt.value

        while iterations < budget:
            iterations += 1
            print(f"\n---\nStarting iteration {iterations} at temp {temp}...")

            # Start with the system message
            messages = [
                {"role": "system", "content": system_message}
            ]

            # Add the training data to the messages
            if training_data:
                messages.append({"role": "assistant", "content": "Previously, we have tested these formulations with the following result:\n" + "\n".join(training_data)})
                # Add the iteration prompt
                messages.append({"role": "user", "content": iterate_prompt.value})
                
            if conversation_dropdown.value == 'Complete':
                print("--- Conversation History ---")
                for msg in messages:
                    print(f"{msg['role']}: {msg['content']}")

            valid_solution = False
            while not valid_solution:
                if enable_TVDL_strategy == 1:  # Assuming '1' means enabled
                    assistant_response = run_TVDL_strategy(system_message, training_data,iterate_prompt.value,context_prompt.value, conversation_dropdown)
                else:
                    response = call_openai_api(messages, temp)
                    assistant_response = response.choices[0].message 
                    
                if assistant_response.role == "assistant":
                    print('Model Response: ', assistant_response.content)
                    
                    suggested_solution = parse_solution(assistant_response.content)

                    if suggested_solution is not None:
                        (lab_result,TrainingDat) = find_matching_result(formulation_df, suggested_solution)
                        if lab_result:
                            current_strength = lab_result
                            print(TrainingDat)
                            training_data.append(f"{TrainingDat} resulted in a strength of {current_strength} MPa.")
                            #print('######TrainingData',training_data)
                            valid_solution = True
                            
                            print('The suggested formulation achieved a strength of ', current_strength, ' MPa.')
                        else:
                            print(f"Iteration {iterations}: No matching lab result found for suggestion {suggested_solution}")
                    else:
                        print(f"Iteration {iterations}: Assistant's response did not contain a valid solution. Trying again.")

                    if not valid_solution:
                        messages[-1]["content"] = iterate_prompt.value + "\nRemember the exact parameter grid: Powderkg: {360, 370, 380, 390,400, 410, 420, 430, 440, 450}, wc: {0.45, 0.5, 0.55, 0.6}, materials: {0.7/0.3, 0.6/0.4, 0.5/0.5}, curing: {ambient, heat}. Please stick to these values and the following format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'"
                else:
                    print(f"Iteration {iterations}: Response not from 'assistant'. Trying again.")

            if current_strength >= desired_strength:
                print(f"\nDesired compressive strength of {desired_strength} MPa achieved after {iterations} iterations. The solution is {suggested_solution}.")
                break

        timestamp = str(int(time.time()))

        # create the file name
        filename = f"Results/LLM/{model_dropdown.value}_{prompt_dropdown.value}_prompt_experiment_{experiment+1}_temp_{temp}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_recursive_{enable_TVDL_strategy}_{timestamp}.csv"
       
        # open the file in write mode
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)

            # write the headers
            writer.writerow(["Formulation", "Compressive Strength"])

            # iterate over the training data
            for data in training_data:
                try:
                    # parse the data to extract formulation and compressive strength
                    formulation, strength_str = data.split(" resulted in a strength of ")
                    strength = float(strength_str.split(" ")[0])  # convert string to float
                    writer.writerow([formulation, strength])
                except ValueError as e:
                    print(f"Failed to parse data: {data}")
                    print(f"Error: {e}")

        print(f"Data for experiment {experiment+1} and temp {temp} successfully saved to {filename}.")



NameError: name 'temperatures' is not defined